In [2]:
import sys
sys.path.append("../mypkg")
from constants import RES_ROOT, FIG_ROOT, DATA_ROOT

In [3]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm, trange
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from easydict import EasyDict as edict
import torch
import mne

%matplotlib inline
plt.style.use(FIG_ROOT/"base.mplstyle")

In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
from data_utils.eeg_load import txt2labinfo, montage_txt_parse, EEG_data, len2num, seiz_lab_from_row_fn
from data_utils.utils import get_triple, rec_data, digitize_data, convert_to_type
from utils.misc import load_txt, save_pkl, load_pkl

In [7]:
root = list(DATA_ROOT.glob("EEG_ep"))[0]
root

PosixPath('/data/rajlab1/user_data/jin/MyResearch/EEG-sz-det_dev/notebooks/../mypkg/../data/EEG_ep')

# Pre-analysis

Check num of folder in each session

In [6]:
# all session paths
# count number of folder in each session
nums = []
for p in list(root.rglob("s0*")):
    num = len(list(p.glob("*")))
    nums.append(num)
    if num > 1:
        print(p)
    
nums = np.array(nums)
print((nums==2).sum())
print(np.max(nums))
# all session only have one data

0
1


# Output `csv` file 



In [36]:
names =["dataset", "sub", "session", "montage", "file_stem", "relative_path"]
working_set = "00_epilepsy" # 00_epilepsy
#working_set = "01_no_epilepsy" # 00_epilepsy
working_path = root/working_set
sel_chs_raw = "Fp1, Fp2, F7, F3, Fz, F4, F8, T3, T4, T5, T6, C3, Cz, C4, P3, Pz, P4, O1, O2"
SEL_CHS = [ch.strip().upper() for ch in sel_chs_raw.split(",")];

In [37]:
def path2info(p):
    p_dir = "/".join(str(p).split("/")[-5:]).split('.')[0]
    return str(p.parent).split("/")[-4:] + [p.stem] + [p_dir] 

In [38]:
all_infos = []
for p in tqdm(working_path.rglob("*.edf")):
    all_infos.append(path2info(p))
df = pd.DataFrame(all_infos, columns=names);

1785it [00:00, 3644.29it/s]


In [39]:
# save the ch names for each data in csv
from tqdm import trange
all_chs = []
durs = []
for ix in trange(len(df)):
    dat_path = (root/df["relative_path"].iloc[ix]).with_suffix(".edf")
    dat = mne.io.read_raw_edf(dat_path, verbose=False)
    dur = dat.times[-1]- dat.times[0]
    ch_names_simple = [ch_name.split(" ")[-1].split("-")[0].strip() 
                       for ch_name in dat.ch_names];
    all_chs.append(';'.join(ch_names_simple))
    durs.append(dur)
    #diff_chs = np.setdiff1d(SEL_CHS, ch_names_simple)

100%|██████████████████████████████████████| 1785/1785 [00:10<00:00, 170.60it/s]


In [40]:
df["all_chs"] = all_chs
df["total_dur"] = durs
df["is_ep"] = True if working_set.startswith("00") else False
df.to_csv(root/f"all_data_{working_set}.csv");

# Split data

In [31]:
# get small data to test
all_ep = pd.read_csv(DATA_ROOT/"EEG_ep/all_data_00_epilepsy.csv", index_col=0);
all_noep = pd.read_csv(DATA_ROOT/"EEG_ep/all_data_01_no_epilepsy.csv", index_col=0);
len(all_ep), len(all_noep)
n_half = 100

np.random.seed(0)
noep_idxs = np.random.choice(len(all_noep), size=n_half, replace=False)
ep_idxs = np.random.choice(len(all_ep), size=n_half, replace=False)
subset_noep= all_noep.copy().iloc[noep_idxs]
subset_ep = all_ep.copy().iloc[ep_idxs]

subset_ep.index = np.arange(n_half)
subset_noep.index = np.arange(n_half);
subset_ep.to_csv(root/f"all_data_00_epilepsy_small.csv");
subset_noep.to_csv(root/f"all_data_01_no_epilepsy_small.csv");

# Preprocess 

In [35]:
dataset = EEG_data("01_no_epilepsy_small", "ALL", 
                   root=DATA_ROOT/"EEG_ep",
                   preprocess_dict=dict(is_detrend=True, 
                                      is_drop=True,
                                      target_fs=90, 
                                      filter_limit=[1, 45], 
                                      is_diff=False)
                  );

In [36]:
from tqdm import trange
for ix in trange(len(dataset.all_data)):
    dataset.get_preprocess_data(ix, regen=False)

100%|███████████████████████████████████████████| 79/79 [01:15<00:00,  1.05it/s]
